In [1]:
import json
import sys
import os

from easydict import EasyDict as edict
import pymysql as sql

## 하위 폴더에 있는 패키지 로딩을 위한 밑작업
SEP       = os.path.sep
MISC_PATH = SEP.join(os.getcwd().split(SEP)[:-2])
ROOT_PATH = SEP.join(os.getcwd().split(SEP)[:-4])
sys.path.append(MISC_PATH)

from misc.config import *
from misc.utils import *

[INFO] config.json 파일을 로딩합니다.
[INFO] ports.json 파일을 로딩합니다.


In [2]:
host   = CONFIGS.global_host
port   = PORTS.sql_port
user   = CONFIGS.sql_user
passwd = CONFIGS.sql_passwd

In [3]:
## DB 서버에 연결
conn   = sql.connect(host     = host, user = user, port = port,
                     password = passwd, db = 'study')

cursor = conn.cursor()

## **1. 삽입**
### 1-1. INSERT
~~~SQL
    INSERT INTO [테이블] ([필드 목록]) VALUES ([값 목록]);
~~~
- SQL Server나 Maria DB에서는 INTO를 생략하는 것을 허용있다.
    - 오라클을 포함한 일부 DBMS는 INTO를 생략하면 에러를 발생한다.
    - SQL 표준에는 INTO가 필수로 되어 있어 같이 사용하는 것이 좋다.
 
- 모든 필드에 값을 추가할 때는 필드 목록을 생략할 수 있다.
~~~SQL
    INSERT INTO [테이블] VALUES ([값 목록]);
~~~

- SQL Server와 MariaDB는 이름순으로 정렬하여 춘천과 홍천 사이에 평택이 삽입된다.
- 필드 목록을 명시하지 않는 경우에는 미리 선언한 필드 목록의 순서와 동일하게 데이터를 넣어야한다.

In [4]:
values  = [['"논산"', '554', '12', '"n"', '"충청"']]
columns = 'NAME, AREA, POPU, METRO, REGION'
insert_(cursor, 'tCity', column = columns, values = values)
select(cursor, '논산 데이터 삽입', table_name = 'tCity')

values  = [['"평택"', '453', '51', '"n"', '"경기"']]
insert_(cursor, 'tCity', values)
select(cursor, '평택 데이터 삽입', table_name = 'tCity')

[query] insert into tCity (NAME, AREA, POPU, METRO, REGION) values ("논산", 554, 12, "n", "충청");
[INFO] 데이터 삽입 완료 

[query] select * from tCity;
[논산 데이터 삽입]
('논산', 554, 12, 'n', '충청')
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')
('순천', 910, 27, 'n', '전라')
('오산', 42, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')


[query] insert into tCity values ("평택", 453, 51, "n", "경기");
[INFO] 데이터 삽입 완료 

[query] select * from tCity;
[평택 데이터 삽입]
('논산', 554, 12, 'n', '충청')
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')
('순천', 910, 27, 'n', '전라')
('오산', 42, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('평택', 453, 51, 'n', '경기')
('홍천', 1819, 7, 'n', '강원')




### 1-2. COMMIT, ROLLBACK
- DB에 데이터를 삽입하면 commit을 하기 전에는 임시 영역에서 삽입 대기 상태로 남아있다.
    - 임시 영역에 변경 사항이 이상 없는 경우 테이블에 적용하는 commit 시킨다.
    - 데이터를 잘 못 넣었을 경우 rollback을 통해 변경 사항을 취소할 수 있다.
 
~~~SQL
    - COMMIT;
    - ROLLBACK;
~~~
- pymysql에서는 conn.commit()을 통해 commit시켜 준다.

##### **연습 문제 #001.** 

> Q1. 도시 목록에 용인을 삽입하라. 면적 293에 인구 98만이되 데이터는 임의값을 써도 상관없다.  
> Q2. 직원 목록에 자신의 신상을 삽입해 보아라.

In [5]:
values = [['"용인"', '293', '98', '"n"', '"경기"']]
insert_(cursor, 'tCity', values)

values = [['"김둘기"', '"개발팀"', '"남"', '"2023-08-16"', '"대리"', '999', '89.80']]
insert_(cursor, 'tStaff', values)

[query] insert into tCity values ("용인", 293, 98, "n", "경기");
[INFO] 데이터 삽입 완료 

[query] insert into tStaff values ("김둘기", "개발팀", "남", "2023-08-16", "대리", 999, 89.80);
[INFO] 데이터 삽입 완료 



In [6]:
select(cursor, "[A1. Result]", table_name = 'tCity')
select(cursor, "[A2. Result]", table_name = 'tStaff')

[query] select * from tCity;
[[A1. Result]]
('논산', 554, 12, 'n', '충청')
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')
('순천', 910, 27, 'n', '전라')
('오산', 42, 21, 'n', '경기')
('용인', 293, 98, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('평택', 453, 51, 'n', '경기')
('홍천', 1819, 7, 'n', '강원')


[query] select * from tStaff;
[[A2. Result]]
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 320, Decimal('56.00'))
('김둘기', '개발팀', '남', datetime.date(2023, 8, 16), '대리', 999, Decimal('89.80'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '사원', 315, Decimal('45.10'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 400, Decimal('92.00'

### 1-3. 확장 INSERT 문
- 여러개의 필드를 한 번에 삽입하려면 삽입하고자 하는 필드들을 한 번에 명시해주면 된다.
~~~SQL
    INSERT INTO [테이블] (필드명) VALUES ([데이터 1]), ..., ([데이터 n])
~~~
- SQL Server, MariaDB 등 많은 DBMS에서는 지원하지만, ORACLE에서는 지원하지 않는다.

##### **연습 문제 #002.** 

> Q1. 도시 목록에 아래 데이터들을 삽입하라  
> (이천, 461, 21, n, 경기), (대구, 883, 248, y, 경상), (영월, 1127, 4, n, 강원)

In [7]:
values = [
          ['"이천"', '461',  '21', '"n"', '"경기"'],
          ['"대구"', '884', '248', '"y"', '"경상"'],
          ['"영월"', '1127', '4', '"n"', '"강원"']
         ]

insert_(cursor, table_name = 'tCity', values = values)
select(cursor, '[A1. Result]', table_name = 'tCity')

[query] insert into tCity values ("이천", 461, 21, "n", "경기"), ("대구", 884, 248, "y", "경상"), ("영월", 1127, 4, "n", "강원");
[INFO] 데이터 삽입 완료 

[query] select * from tCity;
[[A1. Result]]
('논산', 554, 12, 'n', '충청')
('대구', 884, 248, 'y', '경상')
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')
('순천', 910, 27, 'n', '전라')
('영월', 1127, 4, 'n', '강원')
('오산', 42, 21, 'n', '경기')
('용인', 293, 98, 'n', '경기')
('이천', 461, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('평택', 453, 51, 'n', '경기')
('홍천', 1819, 7, 'n', '강원')




### 1-4. INSERT SELECT 문
- 이미 저장되어 있는 정보를 복사할 때 INSERT SELECT 문을 사용한다.
~~~SQL
    INSERT INTO [타겟 테이블]([필드명]) SELECT ... FROM [원본 테이블];
~~~

- 유사하게 CREATE SELECT문은 새로운 테이블을 만들어 결과셋을 삽입한다.
    - 백업 테이블을 만들어 작업하기 위해 이 구문을 이용할 수 있다.
    - SQL Server는 CREATE SELECT 대신 SELECT INTO 명령을 사용해 복사할 수 있다.
 
~~~SQL
    CREATE TABLE [대상 테이블] AS SELECT [필드 목록] FROM [원본 테이블];
~~~

- 데이터만 복사할 뿐 기본키나 외래키 같은 제약 조건까지 복사하지 않는다.

In [8]:
## 아래 쿼리문은 tCity테이블에서 경기도 지역 도시만으로 테이블을 생성한다.
query = 'create table if not exists tSudo as select name, area, popu from tCity where region = "경기"'

cursor.execute(query)

select(cursor, '[create tSudo]', table_name = 'tSudo')

[query] select * from tSudo;
[[create tSudo]]
('서울', 605, 974)
('오산', 42, 21)
('용인', 293, 98)
('이천', 461, 21)
('평택', 453, 51)




##### **연습 문제 #003.** 

> Q1. 성취도가 80점 이상인 직원만 골라 이름과 월급에 대한 보고서를  
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;별도의 테이블로 작성하라.

In [9]:
query = 'create table if not exists tReport as select name, salary from tStaff where score >= 80'
cursor.execute(query)

select(cursor, '[A1. Result]', table_name = 'tReport')
select(cursor,    '[A1. REF]', table_name = 'tStaff',
       column = 'name, salary, score', cond = 'score >= 80')

[query] select * from tReport;
[[A1. Result]]
('김둘기', 999)
('김유신', 420)
('성삼문', 285)
('신사임당', 400)
('정몽주', 370)


[query] select name, salary, score from tStaff where score >= 80;
[[A1. REF]]
('김둘기', 999, Decimal('89.80'))
('김유신', 420, Decimal('88.80'))
('성삼문', 285, Decimal('87.75'))
('신사임당', 400, Decimal('92.00'))
('정몽주', 370, Decimal('89.50'))




In [10]:
query = 'create table if not exists tCity_bu as select * from tCity'
cursor.execute(query)

select(cursor, '[BACKUP]', table_name = 'tCity_bu')

[query] select * from tCity_bu;
[[BACKUP]]
('논산', 554, 12, 'n', '충청')
('대구', 884, 248, 'y', '경상')
('부산', 765, 342, 'y', '경상')
('서울', 605, 1000, 'y', '충청')
('순천', 910, 27, 'n', '전라')
('영월', 1127, 4, 'n', '강원')
('오산', 42, 42, 'n', '경기')
('용인', 293, 98, 'n', '경기')
('이천', 461, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('평택', 453, 51, 'n', '경기')
('홍천', 1819, 7, 'n', '강원')




In [11]:
query = 'create table if not exists tStaff_bu as select * from tStaff'
cursor.execute(query)

select(cursor, '[BACKUP]', table_name = 'tStaff_bu')

[query] select * from tStaff_bu;
[[BACKUP]]
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 352, Decimal('56.00'))
('김둘기', '개발팀', '남', datetime.date(2023, 8, 16), '대리', 999, Decimal('89.80'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '대리', 315, Decimal('45.10'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 314, Decimal('87.75'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 440, Decimal('92.00'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 407, Decimal('74.20'))
('유관순', '영업부', '여', datetime.date(2009, 3, 1), '과장', 418, None)
('윤봉길', '영업부', '남', datetime.date(2015, 8, 15), '과장', 385, Decimal('71.25'))
('을지문덕', '영업부', '남', datetime.date(2019, 6, 29), 

## **2. 삭제**
### 2-1. DELETE
- 테이블에 저장되어 있는 데이터를 삭제할 때는 DELETE 문을 사용한다.
    - 삭제는 레코드 단위로 수행하므로 열에 대한 지정은 없다.
    - 오라클과 SQL Server는 FROM 생략할 수 있지만, MariaDB는 생략할 수 없다.
 
- 조건에 맞는 데이터를 찾아서 제거하기 위해선 뒤에 WHERE를 추가한다.

~~~SQL
    DELETE FROM [테이블 명] WHERE [조건];
~~~

- DELETE 문에 WHERE절을 추가 하지 않으면 전체 테이블의 데이터가 사라져 조심해야한다.
    - WHERE 문을 명시하더라도, 조건을 잘못 지정할 수 있으므로
      select문으로 먼저 확인하고 delete 문으로 제거하는 것이 좋다. 

In [12]:
## 아래 쿼리문은 지역 이름이 "경기"인 모든 데이터를 제거한다.
delete_(cursor, table_name = 'tCity', cond = 'region = "경기"')
select(cursor, '[경기도 데이터 삭제]', table_name = 'tCity')

[query] delete from tCity where region = "경기";
[INFO] 데이터 제거 완료 

[query] select * from tCity;
[[경기도 데이터 삭제]]
('논산', 554, 12, 'n', '충청')
('대구', 884, 248, 'y', '경상')
('부산', 765, 342, 'y', '경상')
('순천', 910, 27, 'n', '전라')
('영월', 1127, 4, 'n', '강원')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')




##### **연습 문제 #004.** 

> Q1. 영업부 직원을 전부 해고시켜 보자.

In [13]:
delete_(cursor, table_name = 'tStaff', cond = 'depart = "영업부"')
select(cursor, '[A1. Result]', table_name = 'tStaff')

[query] delete from tStaff where depart = "영업부";
[INFO] 데이터 제거 완료 

[query] select * from tStaff;
[[A1. Result]]
('김둘기', '개발팀', '남', datetime.date(2023, 8, 16), '대리', 999, Decimal('89.80'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '사원', 315, Decimal('45.10'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('이사부', '총무부', '남', datetime.date(2000, 2, 3), '대리', 375, Decimal('50.00'))
('이율곡', '총무부', '남', datetime.date(2016, 3, 8), '과장', 385, Decimal('65.40'))
('장보고', '인사과', '남', datetime.date(2005, 4, 1), '부장', 440, Decimal('58.30'))
('정몽주', '총무부', '남', datetime.date(2010, 9, 16), '대리', 370, Decimal('89.50'))
('정약용', '총무부', '남', datetime.date(2020, 3, 14), '과장', 380, Decimal('69.80'))
('허난설헌', '인사과', '여', datetime.date(2020, 1, 5),

### 2-2. TRUNCATE
~~~SQL
    DELETE FROM [테이블 명];
~~~
- 위 쿼리문은 테이블 내 모든 데이터를 제거하지만, 속도가 느리다.

~~~SQL
    TRUNCATE TABLE [테이블 명];
~~~
- 위 쿼리문은 DELETE FROM 처럼 테이블 내 모든 데이터를 제거하지만,  
  속도가 빠르고 로그가 남지 않아 ROLLBACK으로도 복구할 수 없다.
- 테이블까지 제거하는 방법은 DROP TABLE을 사용한다.

In [14]:
## tCity 테이블 내 데이터를 모두 지우고, 서울, 부산 데이터만 추가한다.
cursor.execute('truncate table tCity')
select(cursor, '[BEFORE]', table_name = 'tCity')

values = [
            ['"서울"', '605', '974', '"y"', '"경기"'],
            ['"부산"', '765', '342', '"y"', '"경상"']
         ]
insert_(cursor, table_name = 'tCity', values = values)
select(cursor, '[AFTER]', table_name = 'tCity')

[query] select * from tCity;
[[BEFORE]]


[query] insert into tCity values ("서울", 605, 974, "y", "경기"), ("부산", 765, 342, "y", "경상");
[INFO] 데이터 삽입 완료 

[query] select * from tCity;
[[AFTER]]
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')




## **3. 갱신**
### 3-1. UPDATE
- 테이블에 있는 데이터를 변경할 때는 UPDATE를 사용한다.
    -  조건에 맞는 데이터만 변경하고 싶다면, WHERE 절을 사용한다.
        - WHERE 절로 변경 대상 레코드를 찾고 SET 문의 대입문을 순서대로 실행한다.  
    -  DELETE 문과 마찬가지로 일반적으로 WHERE문을 함께 사용한다.
    -  UPDATE 문도 임시 영역에 저장되며 확정하려면 COMMIT을 사용해야한다.
~~~SQL
    UPDATE [테이블 명] SET 필드=값 [, 필드=값] WHERE [조건];
~~~

- WHERE 절에는 SELECT 문과 동일하게 비교 연산자, BETWEEN, IN, LIKE 등 연산자를 사용할 수 있다.
    - SET 문에는 필드끼리의 연산도 가능하다.

In [15]:
## 아래 쿼리문을 이용하면 도시 이름이 서울인 데이터의 인구수를 1000, 지역을 충청으로 바꿔준다.
update_(cursor, table_name = 'tCity_bu', cond = 'name = "서울"',
        set_ = 'popu = 1000, region = "충청"')

select(cursor, '[데이터 갱신 확인]', table_name = 'tCity_bu')

[query] update tCity_bu set popu = 1000, region = "충청" where name = "서울";
[INFO] 데이터 업데이트 완료 

[query] select * from tCity_bu;
[[데이터 갱신 확인]]
('논산', 554, 12, 'n', '충청')
('대구', 884, 248, 'y', '경상')
('부산', 765, 342, 'y', '경상')
('서울', 605, 1000, 'y', '충청')
('순천', 910, 27, 'n', '전라')
('영월', 1127, 4, 'n', '강원')
('오산', 42, 42, 'n', '경기')
('용인', 293, 98, 'n', '경기')
('이천', 461, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('평택', 453, 51, 'n', '경기')
('홍천', 1819, 7, 'n', '강원')




In [16]:
## 아래 쿼리문은 오산의 인구를 두 배로 늘려준다.
update_(cursor, table_name = 'tCity_bu', cond = 'name = "오산"',
        set_ = 'popu = popu * 2')

select(cursor, '[데이터 갱신 확인]', table_name = 'tCity_bu')

[query] update tCity_bu set popu = popu * 2 where name = "오산";
[INFO] 데이터 업데이트 완료 

[query] select * from tCity_bu;
[[데이터 갱신 확인]]
('논산', 554, 12, 'n', '충청')
('대구', 884, 248, 'y', '경상')
('부산', 765, 342, 'y', '경상')
('서울', 605, 1000, 'y', '충청')
('순천', 910, 27, 'n', '전라')
('영월', 1127, 4, 'n', '강원')
('오산', 42, 84, 'n', '경기')
('용인', 293, 98, 'n', '경기')
('이천', 461, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('평택', 453, 51, 'n', '경기')
('홍천', 1819, 7, 'n', '강원')




##### **연습 문제 #005.** 

> Q1. 여자 사원급을 모두 대리로 진급시켜라.  
> Q2. 영업부 직원의 월급을 10%씩 인상하라.

In [17]:
update_(cursor, table_name = 'tStaff_bu', cond = 'gender = "여" and grade = "사원"',
        set_ = 'grade = "대리"')

select(cursor, '[A1. Result]', table_name = 'tStaff_bu')

[query] update tStaff_bu set grade = "대리" where gender = "여" and grade = "사원";
[INFO] 데이터 업데이트 완료 

[query] select * from tStaff_bu;
[[A1. Result]]
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 352, Decimal('56.00'))
('김둘기', '개발팀', '남', datetime.date(2023, 8, 16), '대리', 999, Decimal('89.80'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '대리', 315, Decimal('45.10'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 314, Decimal('87.75'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 440, Decimal('92.00'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 407, Decimal('74.20'))
('유관순', '영업부', '여', datetime.date(2009, 3, 1), '과장', 418, None)
('윤봉길', '영업부', '남', da

In [18]:
update_(cursor, table_name = 'tStaff_bu', cond = 'depart = "영업부"',
        set_ = 'salary = salary * 1.1')

select(cursor, '[A2.]', table_name = 'tStaff_bu')

[query] update tStaff_bu set salary = salary * 1.1 where depart = "영업부";
[INFO] 데이터 업데이트 완료 

[query] select * from tStaff_bu;
[[A2.]]
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 387, Decimal('56.00'))
('김둘기', '개발팀', '남', datetime.date(2023, 8, 16), '대리', 999, Decimal('89.80'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '대리', 315, Decimal('45.10'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 345, Decimal('87.75'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 484, Decimal('92.00'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 448, Decimal('74.20'))
('유관순', '영업부', '여', datetime.date(2009, 3, 1), '과장', 460, None)
('윤봉길', '영업부', '남', datetime.date(2

In [19]:
conn.commit()
conn.close()

### 3-2. CRUD (Create Read Update Delete)
- 데이터를 관리하는 명령이라 SQL 명령어 분류상 DML에 속한다.

# **99. 참고자료**
## **99-1. 도서** 
- 소문난 명강의 - 김상형의 SQL 정복 | 김상형 저 / 한빛 미디어

## **99-2. 논문, 학술지**

## **99-3. 웹 사이트**

## **99-4. 데이터셋 출처**
